In [ ]:
import psycopg2
import sqlalchemy
import os 
import pandas as ps
import numpy as np
from trueskill import Rating
import globalConstants as gConst

In [ ]:
PARENT_FOLDER = gConst.PARENT_FOLDER
GSV_CSV = PARENT_FOLDER + "AllFinal.csv"
PSP_CSV = "PSPNet_training_Apr7_20.csv"
CAR_CSV = PARENT_FOLDER + "carImageData/carData_Apr1_20.csv"
OUTCOME_LABELS = gConst.OUTCOME_LABELS
HTTP_BASE_STRING = gConst.HTTP_SITE
SELECTED_IMAGE_FOLDER = gConst.SELECTED_IMAGE_FOLDER
PSP_ORIG_NAMES = gConst.PSP_ORIG_NAMES
PSP_RENAMED = gConst.PSP_RENAMED
PSP_CATEGORIES = gConst.PSP_CATEGORIES

In [ ]:
def getExampleImageNames(exampleImageFolder):
    imageNames = os.listdir(exampleImageFolder)
    imageIds = []
    for imageName in imageNames:
        imageIds.append(imageName[0:len(imageName)-4])
    #imageDF = ps.DataFrame(imageNames, columns = ['panid'])
    return(imageIds)

In [ ]:
def loadGSVData(gsvFile):
    panids = []
    gsvData = ps.read_csv(gsvFile)
    for index in range(len(gsvData['id'])):
        tempRecord = gsvData.iloc[index]
        panids.append(tempRecord['id'] + "_" + str(tempRecord['GSVHeading']))
    gsvData['panid'] = panids
    return(gsvData)

In [ ]:
def loadImageData(gsvFile,pspFile,testing=False):
    gsvData = loadGSVData(gsvFile)
    if(testing):
        sampledImages = getExampleImageNames(SELECTED_IMAGE_FOLDER)
        gsvData = gsvData[gsvData['panid'].isin(sampledImages)]
    pspData = ps.read_csv(pspFile)
    result = ps.merge(gsvData, pspData, on='panid')
    return(result)

In [ ]:
from sqlalchemy import create_engine 
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432' 
POSTGRES_USERNAME = gCONST.PGUSER
POSTGRES_PASSWORD = gCONST.PGPWORD
POSTGRES_DBNAME = gCONST.PGDB
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))
cnx = create_engine(postgres_str) 
connection = cnx.connect()

In [ ]:
def insertImgMetaRecord(currRecord):
    image_id = str(currRecord['panid'])
    year = str(currRecord['year'])
    month = str(currRecord['month'])
    latitude = str(currRecord['lat'])
    longitude = str(currRecord['lon'])
    http = str(HTTP_BASE_STRING + image_id + ".jpg")
    urban_name = str(currRecord['NAME10'])
    sample_cat = str(currRecord['sampleCat'])
    view_cat = "straight"
    if(currRecord['viewCat'] == "sideAngle"):
        view_cat = "side"
    namesString = 'INSERT INTO image_meta (image_id,year,month,latitude,longitude,view_cat,http,urban_name,sample_cat) '
    valuesString = 'VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    insertString = namesString + valuesString
    try:
        trans = connection.begin()
        a = connection.execute(
            insertString,
            (
                image_id,
                year,
                month,
                latitude,
                longitude,
                view_cat,
                http,
                urban_name,
                sample_cat
            )
        )
        trans.commit()
    except Exception as e:
        print(str(e))
        trans.rollback()

In [ ]:
def fillCarTable(CAR_CSV):
    rawData = ps.read_csv(CAR_CSV)
    numObs = len(rawData['imgId'])
    insertString = 'INSERT INTO car_images (image_id,car_cat) VALUES (%s,%s) '
    print(numObs)
    for recordIndex in range(numObs):
        currRecord = rawData.iloc[recordIndex]
        print('a')
        try:
            trans = connection.begin()
            a = connection.execute(
                insertString,
                (
                    str(currRecord['imgId']),
                    str(currRecord['hasCars'])
                )
            )
            trans.commit()
        except Exception as e:
            print('oy')
            print(str(e))
            trans.rollback()

In [ ]:
def insertImgPSPRecord(currRecord): 
    catString = 'INSERT INTO psp_net_scores (image_id'
    valueString = 'VALUES (%s'
    valArray = [str(currRecord['panid'])]
    for cat in PSP_CATEGORIES:
        catString = catString + ',' + cat
        valueString += ',%s'
        if cat in PSP_RENAMED:
            valArray.append(currRecord[PSP_ORIG_NAMES[PSP_RENAMED.index(cat)]])
        else:
            valArray.append(currRecord[cat])
    catString +=') '
    valueString += ')'
    insertString = catString + valueString
    try:
        trans = connection.begin()
        a = connection.execute( 
            insertString,
            (
                valArray
            )
        )
        trans.commit()
    except Exception as e:
        print(str(e))
        #print("oy")
        trans.rollback()

In [ ]:
def loadImgMetaToSQL(imgRecords):
    numRecords = len(imgRecords['id'])
    for recordIndex in range(numRecords):
        insertImgMetaRecord(imgRecords.iloc[recordIndex])

In [ ]:
def loadImagePSPToSQL(imgRecords):
    numRecords = len(imgRecords['id'])
    for recordIndex in range(numRecords):
        insertImgPSPRecord(imgRecords.iloc[recordIndex])

In [ ]:
def createCategory(inData,categories,categoryName):
    tempData = np.zeros((len(inData['wall']),1))
    index=0
    for category in categories:
        print(category)
        tempData = np.add(tempData,np.array(inData[category]).reshape(len(inData['wall']),1))
    inData[categoryName] = tempData

In [ ]:
def createCategories(inData):
    # crate the built environment category
    builtEnv = ['wall','building','road','windowpane','sidewalk','hovel','house','fence','railing',
               'signboard','skyscraper','path','stairs','runway','screen door','stairway','bridge',
               'bench','booth','awning','streetlight','television receiver','pole','bannister','escalator',
               'fountain','swimming pool','step','sculpture','traffic light','pier']
    createCategory(inData,builtEnv,'built_env')

    build2 = ['house','building','skyscraper','windowpane','hovel']
    createCategory(inData,build2,'build2')
    
    # create the accessibility category 
    accessibility = ['sidewalk','escalator','path','stairs','stairway','bench','step']
    createCategory(inData,accessibility,'accessibility')
    
    # create the allNature category
    allNature = ['tree','grass','plant','field','land','flower','water','sea','waterfall','lake','earth',
                'mountain','rock','sky','sand','hill','dirt track']
    createCategory(inData,allNature,'all_nature')
    
    # create the greenspace cateogry 
    greenspace = ['tree','grass','plant','field','flower']
    createCategory(inData,greenspace,'greenspace')
    
    # create the bluespace category 
    bluespace = ['water','sea','waterfall','lake']
    createCategory(inData,bluespace,'bluespace')
    
    # create the otherNature category
    otherNature = ['earth','mountain','rock','sky','sand','hill','dirt track','land']
    createCategory(inData,otherNature,'other_nature')
    
    # create the animate category
    animate = ['person','boat','car','bus','truck','airplane','van','ship','minibike','animal','bicycle']
    createCategory(inData,animate,'animate')

In [ ]:
def initializeMTTSTableRecord(tablename,imgId):
    nameString = 'INSERT INTO ' + tablename + ' (image_id,strong_mu,strong_sigma,mod_mu,mod_sigma,slight_mu,slight_sigma,n_sampled) '
    valueString = 'VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
    insertString = nameString + valueString
    try:
        trans = connection.begin()
        a = connection.execute(
            insertString, 
            (
                imgId,
                '25.0',
                '8.333',
                '25.0',
                '8.333',
                '25.0',
                '8.333',
                '0'
            )
        )
        trans.commit()
    except Exception as e:
        print(str(e))
        trans.rollback()

In [ ]:
def initializeMTTSTable(label,imgIds):
    for img in imgIds:
        initializeMTTSTableRecord('mt_tskill_' + label,img)

In [ ]:
def initializeWebTSTableRecord(tablename,imgId):
    nameString = 'INSERT INTO ' + tablename + ' (image_id,mu,sigma,n_sampled) '
    valueString = 'VALUES (%s,%s,%s,%s)'
    insertString = nameString + valueString
    try:
        trans = connection.begin()
        a = connection.execute(
            insertString, 
            (
                imgId,
                '25.0',
                '8.333',
                '0'
            )
        )
        trans.commit()
    except Exception as e:
        print(str(e))
        #print("oy")
        trans.rollback()

In [ ]:
def initializeWebTSTable(label,imgIds):
    for img in imgIds:
        initializeWebTSTableRecord('web_tskill_' + label,img)

In [ ]:
def initializeTSTables(imgIds):
    for label in OUTCOME_LABELS:
        initializeMTTSTable(label,imgIds)
        initializeWebTSTable(label,imgIds)

In [ ]:
#imgData = loadImageData(GSV_CSV,PSP_CSV,testing=True)
#imgData = ps.read_csv(GSV_CSV)
#createCategories(imgData)
#loadImgMetaToSQL(imgData)
#loadImagePSPToSQL(imgData)
initializeTSTables(imgData['panid'])
fillCarTable(CAR_CSV)